<a href="https://colab.research.google.com/github/Jorge-Maraver/Calculadora_Matrices_Vectores/blob/main/Target_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP INICIAL

In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split, KFold, StratifiedGroupKFold, StratifiedKFold, GroupKFold, cross_validate
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_selector as selector
from sklearn import set_config
from sklearn.base import clone
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from scipy.stats import uniform, randint
from sklearn.feature_selection import GenericUnivariateSelect, f_classif, mutual_info_classif

# Esto hace que los steps de las pipelines sean df de pandas por defecto, sino serían np arrays
set_config(transform_output="pandas")
pd.set_option("display.max_columns", 100)

base_path = "/content/drive/MyDrive/Colab Notebooks/Target churn"
persondf = pd.read_parquet(f"{base_path}/clients.parquet")
persondf

,customerid,gender,paperlessbilling,paymentmethod,children,contract_date,married,birth_date,churn_date,country,zipcode,contract_channel
0,3d40ba9d86cc7b97612628a924cd63bc,Female,Yes,Electronic check,0,2022-12-22,Yes,1975-05-01,NaT,ES,14576,None
1,ce8db6bfb9d902d89f9dc36eb1fc7df9,Male,No,Mailed check,0,2020-03-07,No,1984-04-01,NaT,ES,40071,Phone
2,5cf42e9ba3bfa476bc222bebcf63f1a8,Male,Yes,Mailed check,0,2022-11-07,No,2010-12-07,2023-05-13,ES,45466,None
3,84f664e4dcfa80ce511615bc91ca6692,Male,No,Bank transfer (automatic),0,2019-04-09,No,1994-08-08,NaT,ES,31420,None
4,0467f23d6b9e7891da40ed53339f5725,Female,Yes,Electronic check,0,2022-11-06,No,1973-08-02,2023-08-20,ES,10119,None
...,...,...,...,...,...,...,...,...,...,...,...,...
11264,40603cfaa31765822d1555c74653ea60,Female,Yes,Electronic check,0,2022-12-24,Yes,1995-09-28,NaT,ES,14576,None
11265,4f7479fd8b8cb089cf02bbe3557c7656,Male,Yes,Electronic check,2,2022-12-17,Yes,1992-08-07,2023-06-17,ES,19605,None
11266,70c4cda7ec0c8a1c40056b1da9f75403,Male,No,Bank transfer (automatic),0,2022-11-09,No,2010-04-19,2023-08-02,ES,26622,None
11267,04608afff3c77e7cd2dfb7827fb656b7,Male,Yes,Electronic check,1,2021-09-11,No,1994-08-25,NaT,ES,28496,None


In [ ]:
persondf.describe(include="all")

,customerid,gender,paperlessbilling,paymentmethod,children,contract_date,married,birth_date,churn_date,country,zipcode,contract_channel
count,11269,11238,11242,11241,11269.000000,11236,11235,11269,2958,11269,11264,2753
unique,11269,2,2,4,NaN,NaN,2,NaN,NaN,2,6328,3
top,3d40ba9d86cc7b97612628a924cd63bc,Male,Yes,Electronic check,NaN,NaN,No,NaN,NaN,ES,35016,Phone
freq,1,5683,6626,3747,NaN,NaN,5813,NaN,NaN,11264,6,1072
mean,NaN,NaN,NaN,NaN,0.447067,2020-05-03 19:16:46.051975,NaN,1981-10-16 14:04:23.874345,2023-06-17 02:54:16.795131,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,0.000000,2017-01-01 00:00:00,NaN,1934-01-12 00:00:00,2023-03-01 00:00:00,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,0.000000,2018-06-04 00:00:00,NaN,1969-04-16 00:00:00,2023-04-25 00:00:00,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,0.000000,2020-08-18 00:00:00,NaN,1983-07-28 00:00:00,2023-06-17 00:00:00,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,1.000000,2022-04-24 00:00:00,NaN,1997-08-10 00:00:00,2023-08-10 00:00:00,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,2.000000,2023-01-28 00:00:00,NaN,2011-12-28 00:00:00,2023-09-28 00:00:00,NaN,NaN,NaN


In [ ]:
persondf.fillna(pd.NA).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11269 entries, 0 to 11268
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerid        11269 non-null  object        
 1   gender            11238 non-null  object        
 2   paperlessbilling  11242 non-null  object        
 3   paymentmethod     11241 non-null  object        
 4   children          11269 non-null  int64         
 5   contract_date     11236 non-null  datetime64[us]
 6   married           11235 non-null  object        
 7   birth_date        11269 non-null  datetime64[us]
 8   churn_date        2958 non-null   datetime64[us]
 9   country           11269 non-null  object        
 10  zipcode           11264 non-null  object        
 11  contract_channel  2753 non-null   object        
dtypes: datetime64[us](3), int64(1), object(8)
memory usage: 1.0+ MB


In [ ]:
# Simulamos un conjunto de clientes desconocido

X_holdout = persondf[persondf.customerid.str.endswith(("d", "e", "f"))].copy()
X_train = persondf[~persondf.customerid.str.endswith(("d", "e", "f"))].copy()

# BASELINE PIPELINE

In [ ]:
# PREPROCESS PIPELINE BASE 0 (sin tener en cuenta la imputación en la permanencia)

# Aqui realizamos las transformaciones básicas iniciales que no tengan riesgo de data leak. Por ejemplo merges o cálculos aritméticos de columnas.
def base_transform(df):
  df["age"] = ((datetime.datetime.now() - pd.to_datetime(df.birth_date)).dt.days/365).round(1)
  df["years_contract"] = ((datetime.datetime.now() - pd.to_datetime(df.contract_date)).dt.days/365).round(1)
  df = df.drop(columns=["birth_date", "contract_date"])
  billdf = pd.read_csv(f"{base_path}/billing.csv", sep=";")
  tenuredf = pd.read_json(f"{base_path}/tenure.json", orient="records")
  billdf["billing_date"] = pd.to_datetime(billdf["billing_date"], errors="coerce")
  billdf["month"] = billdf["billing_date"].dt.month
  tenuredf["month"] = tenuredf["date"].dt.month
  df = df.merge(billdf, on="customerid").merge(tenuredf, on=["customerid", "month"], how="left")
  df = df.fillna(pd.NA) # Para unificar todos los posibles nulos a pd.NA // Aun así esto no es mega fiable, se pueden colar nulos raros
  df = df.reset_index(drop=True)
  return df

# Limpiamos columnas innecesarias para preprocess y training. Se puede hacer antes, pero quizá necesitas alguna en el paso anterior...
drop_columns = ["customerid", "month", "date", "billing_date", "churn_date", "country", "zipcode"]
col_remover = ColumnTransformer(
      transformers=[("drop", "drop", drop_columns)],
      remainder="passthrough", # El resto de columnas no seleccionadas pasan hacia adelante
      force_int_remainder_cols=False, # Esto elimina un warning de sklearn
      verbose_feature_names_out=False # No introduce prefijo al nuevo nombre de columna
    )


# Transformaciones específicas por tipo de dato

# Categóricas
cat_transformer = Pipeline(steps=[
    # Ojo con este missing values. En general los imputers sustituyen todo lo que sea np.nan o pd.NA. Si hay python Nones, hay que tenerlo en cuenta.
    ('imputer', SimpleImputer(missing_values=pd.NA, strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop="first", sparse_output=False, handle_unknown='ignore'))
])

# Numéricas no-zero
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=pd.NA, strategy='median')),
    ('scaler', StandardScaler())
])


# Transformer final
transformer = ColumnTransformer(
    transformers=[
        ('num', num_transformer, selector(dtype_exclude="object")),
        ('cat', cat_transformer, selector(dtype_include="object"))
    ],
    verbose_feature_names_out=True # Introduce un prefijo al nuevo nombre de columna para saber de donde viene
)

# Pipeline de preprocesamiento
preprocess_pipeline = Pipeline(steps=[
    ('col', col_remover),
    ('cast', FunctionTransformer(lambda x: x.infer_objects(), validate=False)),
    ('transformer', transformer),
    ('variance_threshold', VarianceThreshold())
    ])

# Hacemos fit transform para ver si todo funciona
X = base_transform(X_train)
preprocess_pipeline.fit_transform(X)

,num__children,num__age,num__years_contract,num__monthlycharges,num__phone_lines,num__tenure_months,num__tenure_penalty,cat__gender_Male,cat__paperlessbilling_Yes,cat__paymentmethod_Credit card (automatic),cat__paymentmethod_Electronic check,cat__paymentmethod_Mailed check,cat__married_Yes,cat__contract_channel_Phone,cat__contract_channel_Store,cat__internetservice_Fiber optic,cat__internetservice_No,cat__onlinesecurity_No internet service,cat__onlinesecurity_Yes,cat__onlinebackup_No internet service,cat__onlinebackup_Yes,cat__deviceprotection_No internet service,cat__deviceprotection_Yes,cat__techsupport_No internet service,cat__techsupport_Yes,cat__streaming_Yes
0,-0.628381,0.355140,-1.329747,-1.134634,-2.042481,0.510970,0.061600,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-0.628381,0.355140,-1.329747,-1.134634,-2.042481,-0.066452,-0.366870,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.628381,0.355140,-1.329747,-1.134634,-2.042481,-0.643874,-0.795340,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.628381,0.355140,-1.329747,-0.947186,-2.042481,-1.221297,-1.223810,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.628381,0.355140,-1.329747,-0.947186,-2.042481,-1.798719,-1.652568,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81926,-0.628381,0.934372,-1.329747,0.144847,-0.486993,-0.066452,-0.106501,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81927,-0.628381,0.934372,-1.329747,0.144847,-0.486993,-0.066452,-0.106501,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81928,-0.628381,0.934372,-1.329747,0.144847,-0.486993,-0.066452,-0.106501,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81929,-0.628381,0.934372,-1.329747,0.144847,-0.486993,-0.066452,-0.106501,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# PREPROCESS PIPELINE BASE 1 (teniendo en cuenta que la imputación en la permanencia debe ser 0)

# Aqui realizamos las transformaciones básicas iniciales que no tengan riesgo de data leak. Por ejemplo merges o cálculos aritméticos de columnas.
def base_transform(df):
  df["age"] = ((datetime.datetime.now() - pd.to_datetime(df.birth_date)).dt.days/365).round(1)
  df["years_contract"] = ((datetime.datetime.now() - pd.to_datetime(df.contract_date)).dt.days/365).round(1)
  df = df.drop(columns=["birth_date", "contract_date"])
  billdf = pd.read_csv(f"{base_path}/billing.csv", sep=";")
  tenuredf = pd.read_json(f"{base_path}/tenure.json", orient="records")
  billdf["billing_date"] = pd.to_datetime(billdf["billing_date"], errors="coerce")
  billdf["month"] = billdf["billing_date"].dt.month
  tenuredf["month"] = tenuredf["date"].dt.month
  df = df.merge(billdf, on="customerid").merge(tenuredf, on=["customerid", "month"], how="left")
  df = df.fillna(pd.NA) # Para unificar todos los posibles nulos a pd.NA // Aun así esto no es mega fiable, se pueden colar nulos raros
  df = df.reset_index(drop=True)
  return df

# Limpiamos columnas innecesarias para preprocess y training. Se puede hacer antes, pero quizá necesitas alguna en el paso anterior...
drop_columns = ["customerid", "month", "date", "billing_date", "churn_date", "country", "zipcode"]
col_remover = ColumnTransformer(
      transformers=[("drop", "drop", drop_columns)],
      remainder="passthrough", # El resto de columnas no seleccionadas pasan hacia adelante
      force_int_remainder_cols=False, # Esto elimina un warning de sklearn
      verbose_feature_names_out=False # No introduce prefijo al nuevo nombre de columna
    )


# Transformaciones específicas por tipo de dato

# Categóricas
cat_transformer = Pipeline(steps=[
    # Ojo con este missing values. En general los imputers sustituyen todo lo que sea np.nan o pd.NA. Si hay python Nones, hay que tenerlo en cuenta.
    ('imputer', SimpleImputer(missing_values=pd.NA, strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop="first", sparse_output=False, handle_unknown='ignore'))
])

# Numéricas no-zero
num_transformer_no_zero = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=pd.NA, strategy='median')),
    ('scaler', StandardScaler())
])

# Numéricas zero
num_transformer_zero = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=pd.NA, strategy='constant', fill_value=0)),
    ('scaler', MinMaxScaler(clip=True)) # Cuando imputamos 0, no tiene mucho sentido hacer standard scaler, porque perdemos la referencia del 0
])

# Transformer final
transformer = ColumnTransformer(
    transformers=[
        ('num_zero', num_transformer_zero, selector(pattern=".*tenure.*", dtype_exclude="object")),
        ('num_no_zero', num_transformer_no_zero, selector(pattern="^(?!.*tenure).*", dtype_exclude="object")), # PROBAR A ELIMINAR ESTO E IMPUTAR MEDIANA A TODO!
        ('cat', cat_transformer, selector(dtype_include="object"))
    ],
    verbose_feature_names_out=False # Introduce un prefijo al nuevo nombre de columna para saber de donde viene
)

# Pipeline de preprocesamiento
preprocess_pipeline = Pipeline(steps=[
    ('col', col_remover),
    ('cast', FunctionTransformer(lambda x: x.infer_objects(), validate=False)),
    ('transformer', transformer),
    ('variance_threshold', VarianceThreshold())
    ])

# Hacemos fit transform para ver si todo funciona
X = base_transform(X_train)
preprocess_pipeline.fit_transform(X)

,tenure_months,tenure_penalty,children,age,years_contract,monthlycharges,phone_lines,gender_Male,paperlessbilling_Yes,paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check,married_Yes,contract_channel_Phone,contract_channel_Store,internetservice_Fiber optic,internetservice_No,onlinesecurity_No internet service,onlinesecurity_Yes,onlinebackup_No internet service,onlinebackup_Yes,deviceprotection_No internet service,deviceprotection_Yes,techsupport_No internet service,techsupport_Yes,streaming_Yes
0,0.428571,0.246122,-0.628381,0.354984,-1.331107,-1.134634,-2.042481,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.357143,0.205106,-0.628381,0.354984,-1.331107,-1.134634,-2.042481,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.285714,0.164091,-0.628381,0.354984,-1.331107,-1.134634,-2.042481,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.214286,0.123075,-0.628381,0.354984,-1.331107,-0.947186,-2.042481,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.142857,0.082031,-0.628381,0.354984,-1.331107,-0.947186,-2.042481,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81926,0.000000,0.000000,-0.628381,0.934223,-1.331107,0.144847,-0.486993,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81927,0.000000,0.000000,-0.628381,0.934223,-1.331107,0.144847,-0.486993,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81928,0.000000,0.000000,-0.628381,0.934223,-1.331107,0.144847,-0.486993,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81929,0.000000,0.000000,-0.628381,0.934223,-1.331107,0.144847,-0.486993,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
preprocess_pipeline.fit_transform(X).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81931 entries, 0 to 81930
Data columns (total 26 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   tenure_months                          81931 non-null  float64
 1   tenure_penalty                         81931 non-null  float64
 2   children                               81931 non-null  float64
 3   age                                    81931 non-null  float64
 4   years_contract                         81931 non-null  float64
 5   monthlycharges                         81931 non-null  float64
 6   phone_lines                            81931 non-null  float64
 7   gender_Male                            81931 non-null  float64
 8   paperlessbilling_Yes                   81931 non-null  float64
 9   paymentmethod_Credit card (automatic)  81931 non-null  float64
 10  paymentmethod_Electronic check         81931 non-null  float64
 11  pa

In [ ]:
# Calculo de la target
# Solo se usa en entrenamiento, por eso se desacopla de la Pipeline
# Reciclamos el uso de base_transform sobre el dataframe original
def get_target(df):
  churn_date = base_transform(df)[["billing_date", "churn_date"]]
  y = (pd.to_datetime(churn_date.churn_date) - pd.to_datetime(churn_date.billing_date)).dt.days <= 100
  return y.astype(int)

y = get_target(X_train)
y.value_counts()

,count
0,72153
1,9778


# PRIMER EXPERIMENTO SOBRE EL BASELINE

In [ ]:
# Pipeline principal

pipe = Pipeline(steps=[
    ("preprocess", preprocess_pipeline),
    ("clf", LogisticRegression())
])

# Hacemos una prueba de entrenamiento + score, para ver que todo funciona y ver el accuracy en situación de overfitting
X = base_transform(X_train)
y = get_target(X_train)
pipe.fit(X, y).score(X, y)

0.8833530653842868

In [ ]:
pipe

Pipeline(steps=[('preprocess',
                 Pipeline(steps=[('col',
                                  ColumnTransformer(force_int_remainder_cols=False,
                                                    remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['customerid',
                                                                    'month',
                                                                    'date',
                                                                    'billing_date',
                                                                    'churn_date',
                                                                    'country',
                                                                    'zipcode'])],
                                                    verbose_feature_names_out=False)),
                                 ('cast',
                                  FunctionTransformer(func=<function <lambda> at 0x7e48699a4400>)),
                                 ('transform...
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(missing_values=<NA>,
                                                                                                  strategy='most_frequent')),
                                                                                   ('onehot',
                                                                                    OneHotEncoder(drop='first',
                                                                                                  handle_unknown='ignore',
                                                                                                  sparse_output=False))]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7e4869961610>)],
                                                    verbose_feature_names_out=False)),
                                 ('variance_threshold', VarianceThreshold())])),
                ('clf', LogisticRegression())])

In [ ]:
X

,customerid,gender,paperlessbilling,paymentmethod,children,married,churn_date,country,zipcode,contract_channel,age,years_contract,billing_date,monthlycharges,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,phone_lines,streaming,month,date,tenure_months,tenure_penalty
0,3d40ba9d86cc7b97612628a924cd63bc,Female,Yes,Electronic check,0,Yes,NaT,ES,14576,<NA>,49.9,2.3,2023-01-31,29.85,DSL,No,Yes,No,No,0,No,1,2023-01-31,6.0,89.17
1,3d40ba9d86cc7b97612628a924cd63bc,Female,Yes,Electronic check,0,Yes,NaT,ES,14576,<NA>,49.9,2.3,2023-02-28,29.85,DSL,No,Yes,No,No,0,No,2,2023-02-28,5.0,74.31
2,3d40ba9d86cc7b97612628a924cd63bc,Female,Yes,Electronic check,0,Yes,NaT,ES,14576,<NA>,49.9,2.3,2023-03-31,29.85,DSL,No,Yes,No,No,0,No,3,2023-03-31,4.0,59.45
3,3d40ba9d86cc7b97612628a924cd63bc,Female,Yes,Electronic check,0,Yes,NaT,ES,14576,<NA>,49.9,2.3,2023-04-30,35.82,DSL,No,Yes,No,No,0,No,4,2023-04-30,3.0,44.59
4,3d40ba9d86cc7b97612628a924cd63bc,Female,Yes,Electronic check,0,Yes,NaT,ES,14576,<NA>,49.9,2.3,2023-05-31,35.82,DSL,No,Yes,No,No,0,No,5,2023-05-31,2.0,29.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81926,7aea14389aa3d73fd383221574f76778,Female,Yes,Electronic check,0,No,2023-08-18,ES,11510,<NA>,61.0,2.3,2023-04-30,70.60,Fiber optic,No,No,No,No,1,No,4,NaT,NaN,NaN
81927,7aea14389aa3d73fd383221574f76778,Female,Yes,Electronic check,0,No,2023-08-18,ES,11510,<NA>,61.0,2.3,2023-05-31,70.60,Fiber optic,No,No,No,No,1,No,5,NaT,NaN,NaN
81928,7aea14389aa3d73fd383221574f76778,Female,Yes,Electronic check,0,No,2023-08-18,ES,11510,<NA>,61.0,2.3,2023-06-30,70.60,Fiber optic,No,No,No,No,1,No,6,NaT,NaN,NaN
81929,7aea14389aa3d73fd383221574f76778,Female,Yes,Electronic check,0,No,2023-08-18,ES,11510,<NA>,61.0,2.3,2023-07-31,70.60,Fiber optic,No,No,No,No,1,No,7,NaT,NaN,NaN


In [ ]:
X.groupby(["customerid", "month"]).size().sort_values()

customerid                        month
ffe815c8b04b06623d58b4f92c485578  3        1
                                  2        1
                                  1        1
ffe70b20bb6bcfdf0c18af450e520f2a  10       1
                                  9        1
                                          ..
fffa6e797408af052858477b24391591  7        1
                                  8        1
                                  9        1
                                  10       1
000aa74f2769e670325b6e7e76805954  1        1
Length: 81931, dtype: int64

In [ ]:
# Ahora veamos qué tal generaliza el baseline
# Aquí es muy importante decidir si hacemos Cross-Validation (y qué estrategia) o un train-test split fijo

# En este caso tenemos doble casuística:
# - Los registros NO son independientes, dependen del customer id
# - El problema está desbalanceado, necesitamos garantizar proporciones en el target

X = base_transform(X_train)
y = get_target(X_train)

for cv in (KFold(5), StratifiedKFold(5), GroupKFold(5), StratifiedGroupKFold(5)):
  print(f"Cross-Validation {cv.__class__.__name__}")
  cv_score = cross_validate(clone(pipe), X, y, groups=X.customerid, scoring=["accuracy", "f1"], cv=cv, return_train_score=True)
  print(f'Accuracy Train: {np.mean(cv_score["train_accuracy"]):.4f} // Test: {np.mean(cv_score["test_accuracy"]):.4f}')
  print(f'F1 score Train: {np.mean(cv_score["train_f1"]):.4f} // Test: {np.mean(cv_score["test_f1"]):.4f}')
  print()

Cross-Validation KFold


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:86: UserWarning: The groups parameter is ignored by KFold
  warnings.warn(


Accuracy Train: 0.8835 // Test: 0.8816
F1 score Train: 0.1897 // Test: 0.1906

Cross-Validation StratifiedKFold


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:877: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


Accuracy Train: 0.8835 // Test: 0.8821
F1 score Train: 0.1900 // Test: 0.1931

Cross-Validation GroupKFold
Accuracy Train: 0.8837 // Test: 0.8826
F1 score Train: 0.1918 // Test: 0.1878

Cross-Validation StratifiedGroupKFold
Accuracy Train: 0.8837 // Test: 0.8835
F1 score Train: 0.1908 // Test: 0.1882



# EXPERIMENTO CON FEATURE ENGINEERING

In [ ]:
def trf_monthly_charges(df, apply_trf=True):

  if not apply_trf:
    return df

  # MUCHO CUIDADO CON ESTE TIPO DE TRANSFORMACIONES. En este caso se puede hacer, porque nuestro holdout son clientes diferentes.
  # Debemos conocer bien cómo va a llegar el dataset de inferencia para hacer este tipo de cosas
  # Si el dataset ya vieinese mergeado con la información de un mes específico, entonces habría mucho riesgo de data leaking

  df = df.sort_values(["customerid", "month"])
  df["increased_rate_diff"] = df.groupby("customerid")["monthlycharges"].diff(1)
  df["increased_rate_pct"] = df.groupby("customerid")["monthlycharges"].pct_change(1, fill_method=None)
  df[["increased_rate_diff", "increased_rate_pct"]] = df[["increased_rate_diff", "increased_rate_pct"]].fillna(0)
  df = df.sort_index()

  return df

def trf_remove_cols(df, apply_trf=True):
  if not apply_trf:
    return df

  df = df.drop(columns=["contract_channel"])
  return df

def trf_reduce_categories(df, apply_trf=True):
  if not apply_trf:
    return df

  for col in ("onlinesecurity", "onlinebackup", "deviceprotection", "techsupport", "streaming"):
    df[col] = (df[col] == "Yes").astype(str)

  return df

In [ ]:
feature_engineering_transformers = [
    ("trf_monthly_charges", FunctionTransformer(trf_monthly_charges, validate=False)),
    ("trf_remove_cols", FunctionTransformer(trf_remove_cols, validate=False)),
    ("trf_reduce_categories", FunctionTransformer(trf_reduce_categories, validate=False)),
]

pipe = Pipeline(steps=[
    *feature_engineering_transformers,
    ("preprocess", preprocess_pipeline),
    ("clf", LogisticRegression())
])

# Hacemos una prueba de entrenamiento + score, para ver que todo funciona y ver el accuracy en situación de overfitting
X = base_transform(X_train)
y = get_target(X_train)
pipe.fit(X, y).score(X, y)

0.8834018869536562

In [ ]:
X = base_transform(X_train)
y = get_target(X_train)
cv = StratifiedGroupKFold(5)
cv_score = cross_validate(pipe, X, y, groups=X.customerid, scoring=["accuracy", "f1"], cv=cv, return_train_score=True)
print(f'Accuracy Train: {np.mean(cv_score["train_accuracy"]):.4f} // Test: {np.mean(cv_score["test_accuracy"]):.4f}')
print(f'F1 score Train: {np.mean(cv_score["train_f1"]):.4f} // Test: {np.mean(cv_score["test_f1"]):.4f}')

Accuracy Train: 0.8835 // Test: 0.8839
F1 score Train: 0.1865 // Test: 0.1892


In [ ]:
pipe

Pipeline(steps=[('trf_monthly_charges',
                 FunctionTransformer(func=<function trf_monthly_charges at 0x7a72581bbb00>)),
                ('trf_remove_cols',
                 FunctionTransformer(func=<function trf_remove_cols at 0x7a72581baa20>)),
                ('trf_reduce_categories',
                 FunctionTransformer(func=<function trf_reduce_categories at 0x7a72581b8fe0>)),
                ('preprocess',
                 Pipeline(steps=[('col',
                                  ColumnTransfo...
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(missing_values=<NA>,
                                                                                                  strategy='most_frequent')),
                                                                                   ('onehot',
                                                                                    OneHotEncoder(drop='first',
                                                                                                  handle_unknown='ignore',
                                                                                                  sparse_output=False))]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7a7259303f50>)],
                                                    verbose_feature_names_out=False)),
                                 ('variance_threshold', VarianceThreshold())])),
                ('clf', LogisticRegression())])

# EXPERIMENTO CON DESBALACEO

In [ ]:
# Probamos con corrección del desbalanceo de LogisticRegression
# Hacemos una prueba de entrenamiento + score, para ver que todo funciona y ver el accuracy en situación de overfitting
X = base_transform(X_train)
y = get_target(X_train)
pipe.set_params(clf__class_weight="balanced").fit(X, y).score(X, y)

0.7350575484248941

In [ ]:
X = base_transform(X_train)
y = get_target(X_train)
cv = StratifiedGroupKFold(5)
cv_score = cross_validate(pipe, X, y, groups=X.customerid, scoring=["accuracy", "f1"], cv=cv, return_train_score=True)
print(f'Accuracy Train: {np.mean(cv_score["train_accuracy"]):.4f} // Test: {np.mean(cv_score["test_accuracy"]):.4f}')
print(f'F1 score Train: {np.mean(cv_score["train_f1"]):.4f} // Test: {np.mean(cv_score["test_f1"]):.4f}')

Accuracy Train: 0.7354 // Test: 0.7341
F1 score Train: 0.4112 // Test: 0.4095


In [ ]:
# Probamos con Imblearn

imb_base_pipe = ImbPipeline(steps=[
    *feature_engineering_transformers,
    *preprocess_pipeline.steps,
    ("clf", LogisticRegression())
])

cv = StratifiedGroupKFold(5)

for resampler in (RandomUnderSampler(), RandomOverSampler(), SMOTE()):
  print(resampler.__class__.__name__)
  imb_pipe = clone(imb_base_pipe)
  imb_pipe.steps.insert(-2, ("resampler", resampler))
  cv_score = cross_validate(imb_pipe, X, y, groups=X.customerid, scoring=["accuracy", "f1"], cv=cv, return_train_score=True)
  print(f'Accuracy Train: {np.mean(cv_score["train_accuracy"]):.4f} // Test: {np.mean(cv_score["test_accuracy"]):.4f}')
  print(f'F1 score Train: {np.mean(cv_score["train_f1"]):.4f} // Test: {np.mean(cv_score["test_f1"]):.4f}')
  print()

RandomUnderSampler
Accuracy Train: 0.7352 // Test: 0.7340
F1 score Train: 0.4113 // Test: 0.4099

RandomOverSampler
Accuracy Train: 0.7357 // Test: 0.7345
F1 score Train: 0.4112 // Test: 0.4098

SMOTE
Accuracy Train: 0.7376 // Test: 0.7361
F1 score Train: 0.4120 // Test: 0.4107



In [ ]:
imb_pipe

Pipeline(steps=[('trf_monthly_charges',
                 FunctionTransformer(func=<function trf_monthly_charges at 0x7a72581bbb00>)),
                ('trf_remove_cols',
                 FunctionTransformer(func=<function trf_remove_cols at 0x7a72581baa20>)),
                ('trf_reduce_categories',
                 FunctionTransformer(func=<function trf_reduce_categories at 0x7a72581b8fe0>)),
                ('col',
                 ColumnTransformer(force_int_remainder_cols=...
                                                                   SimpleImputer(missing_values=<NA>,
                                                                                 strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7a7259d1ed10>)],
                                   verbose_feature_names_out=False)),
                ('resampler', SMOTE()),
                ('variance_threshold', VarianceThreshold()),
                ('clf', LogisticRegression())])

# EXPERIMENTO CON HIPERPARAMETROS

In [ ]:
# Dejamos ya la estructura de pipeline que hemos ido decidiendo

pipe = ImbPipeline(steps=[
    *feature_engineering_transformers,
    *preprocess_pipeline.steps,
    ("resampler", SMOTE()),
    ('passthrough', ColumnTransformer(transformers=[], remainder="passthrough")), # Para que el último step sea transformer
    ("clf", LogisticRegression(max_iter=300))
])


base_grid = {
    "trf_monthly_charges__kw_args": [{"apply_trf": True}, {"apply_trf": False}],
    "trf_remove_cols__kw_args": [{"apply_trf": True}, {"apply_trf": False}],
    "trf_reduce_categories__kw_args": [{"apply_trf": True}, {"apply_trf": False}],
    "resampler__k_neighbors": [3, 5, 7]
}

grid_no_penalty = base_grid | {
    "clf__penalty": [None],
    "clf__solver": ["lbfgs", "newton-cg"],
}

grid_l2 = base_grid | {
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs", "newton-cg"],
    "clf__C": uniform(loc=0.01, scale=10),
}

grid_l1 = base_grid | {
    "clf__penalty": ["l1"],
    "clf__solver": ["liblinear"],  # Only valid solver for l1 penalty
    "clf__C": uniform(loc=0.01, scale=10),
}

# Lista de grids para evitar combinaciones sin sentido
params = [grid_l2, grid_l1, grid_no_penalty]
cv = StratifiedGroupKFold(3)

tuner = RandomizedSearchCV(pipe, params, cv=cv, n_iter=10, scoring="f1", verbose=4, error_score=0)
tuner.fit(X, y, groups=X.customerid) # En los Hiperparameter tuning si se hace group kfold los groups se pasan en el fit

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END clf__penalty=None, clf__solver=lbfgs, resampler__k_neighbors=5, trf_monthly_charges__kw_args={'apply_trf': False}, trf_reduce_categories__kw_args={'apply_trf': True}, trf_remove_cols__kw_args={'apply_trf': True};, score=0.410 total time=   1.9s
[CV 2/3] END clf__penalty=None, clf__solver=lbfgs, resampler__k_neighbors=5, trf_monthly_charges__kw_args={'apply_trf': False}, trf_reduce_categories__kw_args={'apply_trf': True}, trf_remove_cols__kw_args={'apply_trf': True};, score=0.420 total time=   2.6s
[CV 3/3] END clf__penalty=None, clf__solver=lbfgs, resampler__k_neighbors=5, trf_monthly_charges__kw_args={'apply_trf': False}, trf_reduce_categories__kw_args={'apply_trf': True}, trf_remove_cols__kw_args={'apply_trf': True};, score=0.397 total time=   1.8s
[CV 1/3] END clf__penalty=None, clf__solver=newton-cg, resampler__k_neighbors=3, trf_monthly_charges__kw_args={'apply_trf': True}, trf_reduce_categories__kw_args={'a

RandomizedSearchCV(cv=StratifiedGroupKFold(n_splits=3, random_state=None, shuffle=False),
                   error_score=0,
                   estimator=Pipeline(steps=[('trf_monthly_charges',
                                              FunctionTransformer(func=<function trf_monthly_charges at 0x7e48699a5ee0>)),
                                             ('trf_remove_cols',
                                              FunctionTransformer(func=<function trf_remove_cols at 0x7e48699a6980>)),
                                             ('trf_reduce_categories',
                                              FunctionTransf...
                                                                      {'apply_trf': False}]},
                                        {'clf__penalty': [None],
                                         'clf__solver': ['lbfgs', 'newton-cg'],
                                         'resampler__k_neighbors': [3, 5, 7],
                                         'trf_monthly_charges__kw_args': [{'apply_trf': True},
                                                                          {'apply_trf': False}],
                                         'trf_reduce_categories__kw_args': [{'apply_trf': True},
                                                                            {'apply_trf': False}],
                                         'trf_remove_cols__kw_args': [{'apply_trf': True},
                                                                      {'apply_trf': False}]}],
                   scoring='f1', verbose=4)

In [ ]:
print(tuner.best_score_)
print(tuner.best_params_)

0.4100108289898305
{'clf__penalty': None, 'clf__solver': 'lbfgs', 'resampler__k_neighbors': 7, 'trf_monthly_charges__kw_args': {'apply_trf': True}, 'trf_reduce_categories__kw_args': {'apply_trf': True}, 'trf_remove_cols__kw_args': {'apply_trf': False}}


# OTRA OPCIÓN DE HIPERPARAMETER TUNING

In [ ]:
# Dejamos ya la estructura de pipeline que hemos ido decidiendo

pipe = ImbPipeline(steps=[
    *feature_engineering_transformers,
    *preprocess_pipeline.steps,
    ("selector", GenericUnivariateSelect(score_func=f_classif, mode='k_best', param=10)),
    ("resampler", SMOTE()),
    ('passthrough', ColumnTransformer(transformers=[], remainder="passthrough")), # Para que el último step sea transformer
    ("clf", LogisticRegression(max_iter=300))
])

base_grid = {
    "resampler__k_neighbors": [3, 5, 7],
    "trf_remove_cols__kw_args": [{"apply_trf": True}, {"apply_trf": False}],
    "selector__score_func": [f_classif, mutual_info_classif],
    "selector__param": randint(16, 24)
}

grid_no_penalty = base_grid | {
    "clf__penalty": [None],
    "clf__solver": ["lbfgs", "newton-cg"],
}

grid_l2 = base_grid | {
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs", "newton-cg"],
    "clf__C": uniform(loc=0.01, scale=10),
}

# Lista de grids para evitar combinaciones sin sentido
params = [grid_l2, grid_no_penalty]
cv = StratifiedGroupKFold(3)

tuner_pipe = clone(pipe)
tuner_pipe = tuner_pipe.set_params(trf_monthly_charges__kw_args={"apply_trf": True},
                                   trf_remove_cols__kw_args={"apply_trf": True},
                                   trf_reduce_categories__kw_args={"apply_trf": True},
                                   )

tuner = RandomizedSearchCV(tuner_pipe, params, cv=cv, n_iter=2, scoring="f1", verbose=4, error_score=np.nan)
tuner.fit(X, y, groups=X.customerid) # En los Hiperparameter tuning si se hace group kfold los groups se pasan en el fit

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END clf__penalty=None, clf__solver=newton-cg, resampler__k_neighbors=3, selector__param=18, selector__score_func=<function f_classif at 0x7e486c880d60>, trf_remove_cols__kw_args={'apply_trf': True};, score=0.404 total time=   1.9s
[CV 2/3] END clf__penalty=None, clf__solver=newton-cg, resampler__k_neighbors=3, selector__param=18, selector__score_func=<function f_classif at 0x7e486c880d60>, trf_remove_cols__kw_args={'apply_trf': True};, score=0.416 total time=   1.8s
[CV 3/3] END clf__penalty=None, clf__solver=newton-cg, resampler__k_neighbors=3, selector__param=18, selector__score_func=<function f_classif at 0x7e486c880d60>, trf_remove_cols__kw_args={'apply_trf': True};, score=0.398 total time=   3.1s
[CV 1/3] END clf__penalty=None, clf__solver=lbfgs, resampler__k_neighbors=7, selector__param=19, selector__score_func=<function mutual_info_classif at 0x7e486c89a480>, trf_remove_cols__kw_args={'apply_trf': False};, score

RandomizedSearchCV(cv=StratifiedGroupKFold(n_splits=3, random_state=None, shuffle=False),
                   estimator=Pipeline(steps=[('trf_monthly_charges',
                                              FunctionTransformer(func=<function trf_monthly_charges at 0x7e48699a5ee0>,
                                                                  kw_args={'apply_trf': True})),
                                             ('trf_remove_cols',
                                              FunctionTransformer(func=<function trf_remove_cols at 0x7e48699a6980>,
                                                                  kw_args={'apply_trf': True}))...
                                         'clf__solver': ['lbfgs', 'newton-cg'],
                                         'resampler__k_neighbors': [3, 5, 7],
                                         'selector__param': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e486148ab10>,
                                         'selector__score_func': [<function f_classif at 0x7e486c880d60>,
                                                                  <function mutual_info_classif at 0x7e486c89a480>],
                                         'trf_remove_cols__kw_args': [{'apply_trf': True},
                                                                      {'apply_trf': False}]}],
                   scoring='f1', verbose=4)

In [ ]:
tuner.best_params_

{'clf__penalty': None,
 'clf__solver': 'newton-cg',
 'resampler__k_neighbors': 3,
 'selector__param': 18,
 'selector__score_func': <function sklearn.feature_selection._univariate_selection.f_classif(X, y)>,
 'trf_remove_cols__kw_args': {'apply_trf': True}}

In [ ]:
tuner.best_score_

np.float64(0.40633274613976145)

# PIPELINE FINAL

In [ ]:
# Aplicamos los mejores parámetros a la pipeline
# Ya hemos tomado todas las decisiones con cross-validation, pero para evaluar en holdout podemos usar todo el dataset

final_pipe = clone(pipe)
final_pipe.set_params(**tuner.best_params_).fit(X, y)

X_test = base_transform(X_holdout)
y_test = get_target(X_holdout)
y_pred = final_pipe.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.73      0.83     16492
           1       0.29      0.79      0.43      2357

    accuracy                           0.74     18849
   macro avg       0.63      0.76      0.63     18849
weighted avg       0.88      0.74      0.78     18849



In [ ]:
final_pipe[-1].coef_

array([[-6.17707457,  0.41278422, -0.11216722,  0.0763623 , -0.78390818,
         0.38915791, -0.09441566, -0.03797925,  0.12736486, -0.00645656,
         0.32613908, -0.07949593,  0.31900097, -0.06264262,  0.01981505,
        -0.13912803,  0.05637938,  0.44394004,  0.16164478, -0.37663849,
        -0.51653971, -0.10717429, -0.2118549 , -0.31821139, -0.09890597,
        -0.29338539, -0.47680773,  0.05333511]])